In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import  SVR

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


### 1. Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы


In [3]:
ratings = pd.read_csv('ratings.csv.data')
ratings.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
movies = pd.read_csv('movies.csv.item',encoding='cp1251')
movies.head()

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0



### 2. Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок


In [5]:
ratings.groupby('user_id').rating.count().sort_values(ascending=False).head(1)

user_id
405    737
Name: rating, dtype: int64


### 3. Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь 


In [6]:
movies405user = list(ratings[ratings['user_id'] == 405].item_id)

In [7]:
def movies405(row):
    result = []
    current_rating = 0
    movies405user = list(ratings[ratings['user_id'] == 405].item_id)
    
    for i in range (len(row)):
        if row.loc[i, 'item_id'] in movies405user:
            current_rating = 1
        if row.loc[i, 'item_id'] not in movies405user:
            current_rating = 0
            
        result.append(current_rating)
        
    
    movie405 = pd.Series(result)
    row['movies405'] = movie405
    return row.head()

In [8]:
movies405(ratings)

,user_id,item_id,rating,timestamp,movies405
0,196,242,3,881250949,0
1,186,302,3,891717742,1
2,22,377,1,878887116,1
3,244,51,2,880606923,1
4,166,346,1,886397596,0


In [9]:
ratings405 = ratings[ratings['movies405']==1]

In [10]:
ratings405.head()

,user_id,item_id,rating,timestamp,movies405
1,186,302,3,891717742,1
2,22,377,1,878887116,1
3,244,51,2,880606923,1
6,115,265,2,881171488,1
7,253,465,5,891628467,1


In [11]:
ratings405.head()

,user_id,item_id,rating,timestamp,movies405
1,186,302,3,891717742,1
2,22,377,1,878887116,1
3,244,51,2,880606923,1
6,115,265,2,881171488,1
7,253,465,5,891628467,1


In [12]:
ratings405.columns = ['user_id', 'movie_id', 'rating', 'timestamp', 'movies405']

In [13]:
joined = ratings405.merge(movies, on='movie_id', how='left')
joined.head()

,user_id,movie_id,rating,timestamp,movies405,movie_title,release_date,video_release_date,IMDb_URL,unknown,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,186,302,3,891717742,1,L.A. Confidential (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?L%2EA%2E+Conf...,0,...,0,1,0,0,1,0,0,1,0,0
1,22,377,1,878887116,1,Heavyweights (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Heavyweights%...,0,...,0,0,0,0,0,0,0,0,0,0
2,244,51,2,880606923,1,Legends of the Fall (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Legends%20of%...,0,...,0,0,0,0,0,1,0,0,1,1
3,115,265,2,881171488,1,Hunt for Red October; The (1990),01-Jan-1990,NaN,http://us.imdb.com/M/title-exact?Hunt+for+Red+...,0,...,0,0,0,0,0,0,0,1,0,0
4,253,465,5,891628467,1,Jungle Book; The (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Jungle%20Book...,0,...,0,0,0,0,0,1,0,0,0,0


In [14]:
joined.columns

Index(['user_id', 'movie_id', 'rating', 'timestamp', 'movies405',
       'movie_title', 'release_date', 'video_release_date', 'IMDb_URL',
       'unknown', 'Action', 'Adventure', 'Animation', 'Children's', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'],
      dtype='object')

In [15]:
joined_filtered = joined[['user_id', 'movie_id', 'rating', 
       'movie_title', 'release_date', 'movies405',
       'Action', 'Adventure', 'Animation', "Children's", 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western']]

In [16]:
moviecount = joined_filtered.groupby('movie_id').count().rating
moviecount.reset_index().head()

,movie_id,rating
0,2,131
1,4,209
2,5,86
3,8,219
4,11,236


In [17]:
moviecount.to_csv(r'moviecount.csv')

C:\Program Files (x86)\Soft data analyst\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [18]:
moviecount = pd.read_csv('moviecount.csv', names = ['movie_id', 'rating_count'])
moviecount.head()

,movie_id,rating_count
0,2,131
1,4,209
2,5,86
3,8,219
4,11,236


In [19]:
moviesum = joined_filtered.groupby('movie_id').rating.sum()
moviesum.reset_index().head()

,movie_id,rating
0,2,420
1,4,742
2,5,284
3,8,875
4,11,908


In [20]:
moviesum.to_csv(r'moviesum.csv')

C:\Program Files (x86)\Soft data analyst\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [21]:
moviesum = pd.read_csv('moviesum.csv', names = ['movie_id', 'rating_sum'])
moviesum.head()

,movie_id,rating_sum
0,2,420
1,4,742
2,5,284
3,8,875
4,11,908


In [22]:
joined_filtered.head()

,user_id,movie_id,rating,movie_title,release_date,movies405,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,186,302,3,L.A. Confidential (1997),01-Jan-1997,1,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,22,377,1,Heavyweights (1994),01-Jan-1994,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,244,51,2,Legends of the Fall (1994),01-Jan-1994,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
3,115,265,2,Hunt for Red October; The (1990),01-Jan-1990,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,253,465,5,Jungle Book; The (1994),01-Jan-1994,1,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0


In [23]:
joined_filtered = joined_filtered.merge(moviecount, on='movie_id', how='left')
joined_filtered.head()

,user_id,movie_id,rating,movie_title,release_date,movies405,Action,Adventure,Animation,Children's,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating_count
0,186,302,3,L.A. Confidential (1997),01-Jan-1997,1,0,0,0,0,...,1,0,0,1,0,0,1,0,0,297
1,22,377,1,Heavyweights (1994),01-Jan-1994,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,13
2,244,51,2,Legends of the Fall (1994),01-Jan-1994,1,0,0,0,0,...,0,0,0,0,1,0,0,1,1,81
3,115,265,2,Hunt for Red October; The (1990),01-Jan-1990,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,227
4,253,465,5,Jungle Book; The (1994),01-Jan-1994,1,0,1,0,1,...,0,0,0,0,1,0,0,0,0,85


In [24]:
joined_filtered = joined_filtered.merge(moviesum, on='movie_id', how='left')
joined_filtered.head()

,user_id,movie_id,rating,movie_title,release_date,movies405,Action,Adventure,Animation,Children's,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating_count,rating_sum
0,186,302,3,L.A. Confidential (1997),01-Jan-1997,1,0,0,0,0,...,0,0,1,0,0,1,0,0,297,1236
1,22,377,1,Heavyweights (1994),01-Jan-1994,1,0,0,0,1,...,0,0,0,0,0,0,0,0,13,28
2,244,51,2,Legends of the Fall (1994),01-Jan-1994,1,0,0,0,0,...,0,0,0,1,0,0,1,1,81,280
3,115,265,2,Hunt for Red October; The (1990),01-Jan-1990,1,1,0,0,0,...,0,0,0,0,0,1,0,0,227,877
4,253,465,5,Jungle Book; The (1994),01-Jan-1994,1,0,1,0,1,...,0,0,0,1,0,0,0,0,85,303


In [25]:
joined_filtered.loc[0, 'release_date'][7:11]

'1997'

In [26]:
def year(row):
    result = []
    for i in range(len(joined_filtered)):
        result.append(joined_filtered.loc[i, 'release_date'][7:11])
        
    result = pd.Series(result)
    joined_filtered['year'] = result
    return row.head()

In [27]:
year(joined_filtered)

,user_id,movie_id,rating,movie_title,release_date,movies405,Action,Adventure,Animation,Children's,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating_count,rating_sum,year
0,186,302,3,L.A. Confidential (1997),01-Jan-1997,1,0,0,0,0,...,0,1,0,0,1,0,0,297,1236,1997
1,22,377,1,Heavyweights (1994),01-Jan-1994,1,0,0,0,1,...,0,0,0,0,0,0,0,13,28,1994
2,244,51,2,Legends of the Fall (1994),01-Jan-1994,1,0,0,0,0,...,0,0,1,0,0,1,1,81,280,1994
3,115,265,2,Hunt for Red October; The (1990),01-Jan-1990,1,1,0,0,0,...,0,0,0,0,1,0,0,227,877,1990
4,253,465,5,Jungle Book; The (1994),01-Jan-1994,1,0,1,0,1,...,0,0,1,0,0,0,0,85,303,1994


In [28]:
joined_filtered.tail()

,user_id,movie_id,rating,movie_title,release_date,movies405,Action,Adventure,Animation,Children's,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating_count,rating_sum,year
52857,913,209,2,This Is Spinal Tap (1984),01-Jan-1984,1,0,0,0,0,...,1,0,0,0,0,0,0,191,746,1984
52858,378,78,3,Free Willy (1993),01-Jan-1993,1,0,1,0,1,...,0,0,0,0,0,0,0,33,81,1993
52859,716,204,5,Back to the Future (1985),01-Jan-1985,1,0,0,0,0,...,0,0,0,1,0,0,0,350,1342,1985
52860,276,1090,1,Sliver (1993),01-Jan-1993,1,0,0,0,0,...,0,0,0,0,1,0,0,37,89,1993
52861,12,203,3,Unforgiven (1992),01-Jan-1992,1,0,0,0,0,...,0,0,0,0,0,0,1,182,704,1992


In [29]:
joined_filtered.columns

Index(['user_id', 'movie_id', 'rating', 'movie_title', 'release_date',
       'movies405', 'Action', 'Adventure', 'Animation', 'Children's', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'rating_count', 'rating_sum', 'year'],
      dtype='object')


### 4. Добавьте к датафрейму из задания 3 столбцы:
### По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет, столбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей

In [30]:
df = joined_filtered[['user_id', 'movie_id', 'rating', 
       'Action', 'Adventure', 'Animation', "Children's", 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'rating_count', 'rating_sum', 'year']]
df.head()

,user_id,movie_id,rating,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating_count,rating_sum,year
0,186,302,3,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,297,1236,1997
1,22,377,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,13,28,1994
2,244,51,2,0,0,0,0,0,0,0,...,0,0,1,0,0,1,1,81,280,1994
3,115,265,2,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,227,877,1990
4,253,465,5,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,85,303,1994


### 5. Сформируйте X_train, X_test, y_train, y_test


In [31]:
X = df[['user_id', 'movie_id', 
       'Action', 'Adventure', 'Animation', "Children's", 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'rating_count', 'rating_sum', 'year']]
y = df['rating']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [33]:
sc = StandardScaler()

In [34]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

C:\Program Files (x86)\Soft data analyst\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Program Files (x86)\Soft data analyst\Anaconda\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Program Files (x86)\Soft data analyst\Anaconda\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  


In [35]:
X_train

array([[-0.1871898 ,  0.16503272,  1.59040165, ...,  0.37969917,
         0.38872244, -1.23545096],
       [-0.11925859,  0.63450196, -0.62877198, ..., -0.95818216,
        -0.95014387,  0.57869826],
       [ 0.92990127, -1.00864039, -0.62877198, ...,  0.67795297,
         0.52200325,  0.65428781],
       ...,
       [-0.59477709,  1.0653847 , -0.62877198, ..., -0.71957912,
        -0.75830028,  0.72987736],
       [-0.64006456,  1.24223955, -0.62877198, ..., -0.97522523,
        -0.94408565,  0.72987736],
       [ 1.7828154 ,  0.26793009, -0.62877198, ..., -1.05191907,
        -0.96629912,  0.57869826]])

In [36]:
X_test

array([[ 1.4544812 , -0.62277526, -0.62877198, ..., -0.40428224,
        -0.29787567,  0.27634005],
       [ 0.86574401,  0.2164814 , -0.62877198, ..., -0.01229153,
        -0.18478892, -0.555145  ],
       [ 1.28842712,  0.58948437, -0.62877198, ..., -1.16269905,
        -1.13592924,  0.65428781],
       ...,
       [ 0.48080047, -1.14369319,  1.59040165, ...,  0.8909914 ,
         0.85722468,  0.72987736],
       [-0.88537173, -0.94754508, -0.62877198, ...,  0.88246987,
         0.96425321, -0.2527868 ],
       [ 0.93744918,  0.13930838, -0.62877198, ...,  0.40526379,
         0.56441078, -1.76457782]])

In [37]:
X_train = pd.DataFrame(X_train, columns = X.columns)
X_train.head()

,user_id,movie_id,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating_count,rating_sum,year
0,-0.187190,0.165033,1.590402,-0.435774,-0.212979,-0.309878,1.538144,-0.302207,-0.084668,-0.810172,...,-0.249702,-0.193352,-0.500007,-0.406224,-0.478587,-0.347951,5.446689,0.379699,0.388722,-1.235451
1,-0.119259,0.634502,-0.628772,-0.435774,-0.212979,-0.309878,-0.650134,-0.302207,-0.084668,1.234306,...,-0.249702,-0.193352,-0.500007,-0.406224,2.089484,-0.347951,-0.183598,-0.958182,-0.950144,0.578698
2,0.929901,-1.008640,-0.628772,-0.435774,-0.212979,-0.309878,1.538144,-0.302207,-0.084668,-0.810172,...,-0.249702,-0.193352,1.999970,-0.406224,-0.478587,-0.347951,-0.183598,0.677953,0.522003,0.654288
3,-1.659033,0.177895,-0.628772,-0.435774,-0.212979,-0.309878,-0.650134,-0.302207,-0.084668,-0.810172,...,-0.249702,-0.193352,-0.500007,-0.406224,-0.478587,-0.347951,-0.183598,-1.418345,-1.323734,0.578698
4,-0.957077,1.312981,-0.628772,-0.435774,-0.212979,-0.309878,1.538144,-0.302207,-0.084668,-0.810172,...,-0.249702,-0.193352,-0.500007,-0.406224,-0.478587,-0.347951,-0.183598,-0.728101,-0.687621,0.654288


In [38]:
X_test = pd.DataFrame(X_test, columns = X.columns)

### 6. Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах


In [39]:
treer = DecisionTreeRegressor()
treer

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [40]:
treer.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [41]:
y_test_pred_treer = treer.predict(X_test)

### 7. Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии

In [42]:
mean_absolute_error(y_test, y_test_pred_treer)

0.9948926510924052

In [43]:
mean_squared_error(y_test, y_test_pred_treer)

1.8088527381064976

In [44]:
r2_score(y_test, y_test_pred_treer)

-0.48195966955028147

### Прочее

In [45]:
'''svr = SVR()
svr.fit(X_train, y_train)
y_test_pred_svr = svr.predict(X_test)
mean_squared_error(y_test, y_test_pred_svr)'''

'svr = SVR()\nsvr.fit(X_train, y_train)\ny_test_pred_svr = svr.predict(X_test)\nmean_squared_error(y_test, y_test_pred_svr)'

In [46]:
'''linear = LinearRegression()
linear.fit(X_train, y_train)
y_test_pred_lin = linear.predict(X_test)
mean_squared_error(y_test, y_test_pred_lin)'''

'linear = LinearRegression()\nlinear.fit(X_train, y_train)\ny_test_pred_lin = linear.predict(X_test)\nmean_squared_error(y_test, y_test_pred_lin)'